In [2]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import os
import requests
from tqdm.notebook import tqdm
from datetime import date
import datetime

In [3]:
# Укажите путь к вашему файлу с учетными данными
path_to_credential = 'C:/Users/MerinovDV/Downloads/auto-monitoring-367212-64ec4ad9d3a5.json' 

# Укажите имя таблицы в Google
table_name = 'Auto_monitoring'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_table = gs.open(table_name)

# список листов в Google с данными
google_sheets = ['smart_new', 'laptop_new', 'earphones_new', 'monitor_new', 'processor_new', 'tv_new']


## Парсинг курса с сайта ЦБ

In [4]:
def get_currency(date_parse=None):
    
    """
    возвращает список с датой и курсом доллара
    
    date_parse: str, datetime
        дата, на которую хотим узнать курс доллара
    """
    
    # преобразовываем дату в нужный формат
    if type(date_parse) is type(date.today()):
        date_parse = date_parse.strftime("%d.%m.%Y") 
        
    # если даты нет, то по умолчанию будет сегодняшний день
    elif date_parse == None:
        date_parse = date.today().strftime("%d.%m.%Y")
        
    else:
        date_parse = datetime.datetime.strptime(date_parse, "%d.%m.%Y")
        date_parse = date_parse.strftime("%d.%m.%Y")  

    
    # get запрос на страницу цб с нужной датой
    cb = requests.get(f'https://www.cbr.ru/currency_base/daily/?UniDbQuery.Posted=True&UniDbQuery.To={date_parse}')
    # тип float нужен для того, чтобы данные адекватно загружались в google, а не в формате "текста"
    currency = float(cb.text.split('Доллар США</td>\r\n          <td>')[1].split('</td>')[0].replace(',','.'))
    
    currency_list = [date_parse, currency]
    
    return currency_list

In [53]:
get_currency()

['27.01.2023', 69.1263]

## Добавление курса в таблицу Google

In [5]:
def append_currency():
    
    """
    загружает данные о курсе валют в Google Sheets
    """
    
    # смотрим последнюю дату в таблице
    last_date = work_table.worksheet('currency').col_values(1)[-1]
    last_date = datetime.datetime.strptime(last_date, "%d.%m.%Y")
    
    # считаем разницу в датах
    delta = (datetime.datetime.today() - last_date).days
    
    # если дата не вчерашняя, то заполняем курс за пропущенные даты
    if delta !=1:
        
        for i in range(delta):
            
            last_date = last_date + datetime. timedelta(days=1)
            work_table.worksheet('currency').append_row(get_currency(last_date.strftime("%d.%m.%Y")), 
                                                        value_input_option='USER_ENTERED')
    
    # если дата вчерашняя, то заполняем курс на сегодня
    else:
        work_table.worksheet('currency').append_row(get_currency(), value_input_option='USER_ENTERED')
        
    return print("Данные по курсу ЦБ загружены в Google")


In [6]:
append_currency()

Данные по курсу ЦБ загружены в Google


## Добавление выгрузки в таблицу Google

In [6]:
def append_data(path):
    
    """
    загружает данные по каждой категории товара в Google Sheets
    
    path: str
        путь до excel файлов с выгрузками
    """
    
    spliter = 'yandex-v2-'
    # получаем файлы из папки "Загрузки"
    files = os.listdir(path)
    files_list = []
    for file in files:
        if file.startswith(spliter):
            files_list.append(file)
            
    # получаем даты файлов (выгрузки)   
    dates = []
    for date in files_list:
        split_date = date.split(spliter)[1].split('.xlsx')[0].strip()
        formated_date = pd.to_datetime(split_date, dayfirst=True)
        dates.append(formated_date)
    
    dates.sort()
        
    # проверяем есть ли данная дата в Google таблице
    for f_date in tqdm(dates):
        formated_date = f_date.strftime("%d.%m.%Y")
        cell = work_table.worksheet('smart').find(formated_date) # берем 1 любую таблицу, чтобы проверить дату
        if cell is None:
            excel_df = pd.ExcelFile(path + spliter + formated_date.replace('.', '-') + '.xlsx')
            
            # Если такой даты нет, то пробегаемся по excel файлу и добавляем листы в Google таблицу
            for n, sheet in enumerate(excel_df.sheet_names):
                df = pd.read_excel(excel_df, sheet_name=sheet)
                
                df['Средняя цена, руб'] = (df['Цена min (meta)'] + df['Цена max'])/2
                
                dollar = get_currency(formated_date)[1]
                
                df['Средняя цена, usd'] = df['Средняя цена, руб'] / dollar
                
                if sheet == 'Смартфоны':
                    if 'Производитель.1' in df.columns:
                        df = df.drop(columns=['Производитель.1'], axis=1)
                
                worksheet = work_table.worksheet(google_sheets[n])
                worksheet.append_rows(df.fillna(0).values.tolist(), value_input_option='USER_ENTERED')
                
    return print("Данные загружены в Google")

In [50]:
path = 'C:/Users/MerinovDV/Downloads/'
append_data(path)

  0%|          | 0/46 [00:00<?, ?it/s]

Данные загружены в Google
